In [9]:
# the Python Requests package will allow us to send HTTP requests to get HTML files
import requests

# the GET method indicates that you’re trying to get or retrieve data from a specified resource. 
# to make a GET request, invoke requests.get()
from requests import get

# Beautiful Soup is a Python library for pulling data out of HTML and XML files
from bs4 import BeautifulSoup

# pandas is a Python data analysis library
import pandas as pd

# NumPy is a Python library used for working with large, multi-dimensional arrays and matrices
import numpy as np

from datetime import datetime

import os

In [11]:
url = 'https://speedgreens.co/product-category/'

keys = ['concentrates/hash/', 'cannabis/sativa/']

all_names = []

all_prices = []

all_quantities = []

all_categories = []

date = []

for key in keys:
    results = requests.get(url + key)

    soup = BeautifulSoup(results.content, "html.parser")

    products = soup.find_all('div', class_="product-content")

    for product in products:
        product_link = product.find('a', class_= 'product-loop-title')
        name = product_link.text
        product_page = product_link.get('href')

        product_page_results = requests.get(product_page)
        product_page_soup = BeautifulSoup(product_page_results.content, "html.parser")
        
        price = product_page_soup.find('span', class_='woocommerce-Price-amount amount').text

        quantity = product_page_soup.find('a', class_='filter-item').text
        
        current_month = datetime.now().month
        current_year = datetime.now().year

        all_names.append(name)
        all_prices.append(price)
        all_quantities.append(quantity)
        all_categories.append(key)
        date.append(str(current_month) + '/' + str(current_year))
        
        print(name)
        print(product_page)
        print('Price: ' + price)
        print('Unit: ' + quantity)



M.T.V Hash – Premium AAAA 
https://speedgreens.co/product/mtv-hash-aaaa/
Price: $22.00
Unit: 1g

Red Light Windmill Hash from Amsterdam – Premium AAAA 
https://speedgreens.co/product/red-light-windmill-hash-from-amsterdam-aaaa/
Price: $22.00
Unit: 1g

Romeo & Juliet Hash – Premium AAAA 
https://speedgreens.co/product/romeo-and-juliet-hash-aaaa/
Price: $22.00
Unit: 1g

Two Sword Hash (AA) – Hash Plant 
https://speedgreens.co/product/two-sword-hash-aa/
Price: $8.00
Unit: 1gm

Rockstar Tuna Bubble Hash (AAAA) 
https://speedgreens.co/product/tuna-rockstar-craft-hash-aaaa/
Price: $22.00
Unit: 1gm

Pink Gas Premium Craft Bubble Hash (AAAA) – Hash Weed 
https://speedgreens.co/product/pink-gas-craft-hash-aaaa/
Price: $22.00
Unit: 1gm

Tesla Hash (AAA) – Hash Weed 
https://speedgreens.co/product/tesla-hash-aaa/
Price: $14.00
Unit: 1gm

Diamond Hash (AAA) – Hash Weed 
https://speedgreens.co/product/diamond-hash/
Price: $14.00
Unit: 1gm

Mercedes Hash (AA) – Hashish 
https://speedgreens.co/produ

In [12]:
df = pd.DataFrame(
    {'Date': date,
     'Product': all_names,
     'Category': all_categories,
     'Price': all_prices,
     'Quantity': all_quantities
    })

In [13]:
df

,Date,Product,Category,Price,Quantity
0,3/2022,\nM.T.V Hash – Premium AAAA,concentrates/hash/,$22.00,1g
1,3/2022,\nRed Light Windmill Hash from Amsterdam – Pre...,concentrates/hash/,$22.00,1g
2,3/2022,\nRomeo & Juliet Hash – Premium AAAA,concentrates/hash/,$22.00,1g
3,3/2022,\nTwo Sword Hash (AA) – Hash Plant,concentrates/hash/,$8.00,1gm
4,3/2022,\nRockstar Tuna Bubble Hash (AAAA),concentrates/hash/,$22.00,1gm
5,3/2022,\nPink Gas Premium Craft Bubble Hash (AAAA) – ...,concentrates/hash/,$22.00,1gm
6,3/2022,\nTesla Hash (AAA) – Hash Weed,concentrates/hash/,$14.00,1gm
7,3/2022,\nDiamond Hash (AAA) – Hash Weed,concentrates/hash/,$14.00,1gm
8,3/2022,\nMercedes Hash (AA) – Hashish,concentrates/hash/,$8.00,1gm
9,3/2022,\nZigzag Hash (AAA) – Hashish,concentrates/hash/,$14.00,1gm


In [14]:
output_path = 'speedgreens.csv'

df.to_csv(output_path, mode = 'a', index = False, header = not os.path.exists(output_path))